In [27]:
import numpy as np

# Прямые метода решения СЛАУ

In [28]:
matrix = np.array([[3.8, 6.7, -1.2, 5.2], 
                   [6.4, 1.3, -2.7, 3.8], 
                   [2.4, -4.5, 3.5, -0.6]])

In [29]:
# Реализация с приведением матрицы к треугольной форме а потом к диагональному виду. 
# В последнем столбце будет находиться решение
def makeTriangleNaive(matrix):
    for nrow, row in enumerate(matrix):
        divider = row[nrow]
        row /= divider
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] 
            lower_row -= factor*row
    return matrix

In [30]:
def makeIdentity(matrix):
    for nrow in range(len(matrix)-1,0,-1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            upper_row[-1] -= factor*row[-1]
            upper_row[nrow] = 0
    return matrix

In [31]:
m1 = makeTriangleNaive(np.copy(matrix))
m2 = makeIdentity(m1)
m2

array([[ 1.        ,  0.        ,  0.        ,  0.53344344],
       [-0.        ,  1.        ,  0.        ,  0.49024295],
       [ 0.        ,  0.        ,  1.        ,  0.09309401]])

## Метод выбора ведущего элемента

In [36]:
def makeTrianglePivot(matrix):
    for nrow in range(len(matrix)):
        pivot = nrow + np.argmax(abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow]
        if abs(divider) < 1e-10:
            raise ValueError("Матрица несовместна")
        row /= divider     
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] 
            lower_row -= factor*row
    return matrix

In [37]:
makeTrianglePivot(np.array([[1,0,0,1],
                         [0,0,1,2],
                         [0,1,0,3]
                        ], dtype=float))

array([[1., 0., 0., 1.],
       [0., 1., 0., 3.],
       [0., 0., 1., 2.]])

In [38]:
def gaussSolvePivot(A, b=None):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTrianglePivot(A)
    makeIdentity(A)
    return A[:,-1]

In [39]:
gaussSolvePivot(matrix)

array([0.53344344, 0.49024295, 0.09309401])

## Метод прогонки

In [40]:
###################################################################
### n - число уравнений (строк матрицы)                         ###
### b - диагональ, лежащая над главной (нумеруется: [0;n-2])    ###
### c - главная диагональ матрицы A (нумеруется: [0;n-1])       ###
### a - диагональ, лежащая под главной (нумеруется: [1;n-1])    ###
### f - правая часть (столбец)                                  ###
### x - решение, массив x будет содержать ответ                 ###
###################################################################

def runThrough(a,b,c,f):
    a, b, c, f = tuple(map(lambda k_list: list(map(float, k_list)), (a, b, c, f)))

    alpha = [-b[0] / c[0]]
    beta = [f[0] / c[0]]
    n = len(f)
    print(n)
    x = [0]*n
    print(x)

    for i in range(1, n):
        alpha.append(-b[i]/(a[i]*alpha[i-1] + c[i]))
        beta.append((f[i] - a[i]*beta[i-1])/(a[i]*alpha[i-1] + c[i]))

    x[n-1] = beta[n - 1]

    for i in range(n-1, 0, -1):
        x[i - 1] = alpha[i - 1]*x[i] + beta[i - 1]

    return x

In [41]:
a = [0, 1, 1]
b = [0, 0, 0]
c = [5, 6, 4, -3]
f = [8, 10, 3, -2]

In [42]:
runThrough(a, b, c, f)